# Reservoir Model

The `Reservoir` class models a simple water reservoir. Its primary function is to track water storage based on inflows and outflows. The water level is calculated from the volume and a fixed surface area. This notebook demonstrates how to set up and run a simulation with the `Reservoir` model.

## State Variables

The state of the reservoir is defined by:

- `volume` (float): The current volume of water in the reservoir (m^3).
- `water_level` (float): The water level, calculated as volume divided by surface area (m).
- `outflow` (float): The outflow from the reservoir (m^3/s), determined by downstream demand.

## Parameters

The physical properties of the reservoir are set using:

- `surface_area` (float): The surface area of the reservoir (m^2).

## Simulation Example

The following code simulates a reservoir over several days. It is initialized with a starting volume and receives a constant inflow. The outflow is varied to simulate changing demand. The simulation tracks the reservoir's volume and water level, which are then plotted.

**Note:** To run this notebook, ensure you have `matplotlib` installed (`pip install matplotlib`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the reservoir
initial_state = State(volume=2e7, water_level=0.0, outflow=0.0)
parameters = Parameters(surface_area=2e6) # 2 km^2

# Create a Reservoir instance
reservoir = Reservoir(name="main_reservoir", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 3600  # time step in seconds (1 hour)
simulation_duration = 3600 * 24 * 10  # 10 days
num_steps = int(simulation_duration / dt)
inflow_rate = 20.0  # constant inflow in m^3/s

# Store results for plotting
history = []

# Run the simulation loop
for t in range(num_steps):
    # Simulate changing outflow demand
    if (t // 24) % 2 == 0: # Alternate demand every other day
        requested_outflow = 25.0
    else:
        requested_outflow = 15.0
        
    action = {'outflow': requested_outflow}
    
    reservoir.set_inflow(inflow_rate)
    current_state = reservoir.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", reservoir.get_state())

# Prepare data for plotting
time_in_days = [i * dt / (3600*24) for i in range(num_steps)]
volumes = [s['volume'] / 1e6 for s in history] # in million m^3
water_levels = [s['water_level'] for s in history]

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_in_days, volumes, label='Volume')
ax1.set_ylabel('Volume (million m^3)')
ax1.set_title('Reservoir Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_days, water_levels, label='Water Level', color='orange')
ax2.set_xlabel('Time (days)')
ax2.set_ylabel('Water Level (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
